In [1]:
# This is a development notebook, not meant for demo

In [43]:
from rich import print
from langchain_community.document_loaders import UnstructuredFileLoader
from unstructured.cleaners.core import clean_extra_whitespace, group_broken_paragraphs
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
loaders = [
    UnstructuredFileLoader(
        "/teamspace/studios/this_studio/example_data/2401.08406.pdf",
        post_processors=[clean_extra_whitespace, group_broken_paragraphs],
    ),
    UnstructuredFileLoader(
        "/teamspace/studios/this_studio/example_data/2401.00908.pdf",
        post_processors=[clean_extra_whitespace, group_broken_paragraphs],
    ),
]

In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n\n", "\n\n"],
    chunk_size=1000,
    chunk_overlap=300,
    length_function=len,
    is_separator_regex=False,
)

docs = []
for loader in loaders:
    docs.extend(
        loader.load_and_split(text_splitter=text_splitter),
    )

In [4]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS

In [5]:
model_name = "BAAI/bge-large-en-v1.5"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}  # set True to compute cosine similarity

embeddings_model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

In [6]:
db = FAISS.from_documents(documents=docs, embedding=embeddings_model)

In [7]:
def pretty_print_docs(results, *scores):
    if scores:
        results = zip(results, scores)

    for result in results:
        if isinstance(result, tuple):
            print(result[1])
            print(result[0])
        else:
            print(result)
        print("\n---------\n")

In [8]:
query1 = "Where was the agriculture dataset collected for the USA?"
query2 = "How many pdf data were collected from the USA?"
query3 = "Which contries were used to collect dataset?"
query4 = "What are the metrics used to evaluate the answers?"
query5 = "how was the content and structure of available documents augmented?"
query6 = "What was the answer generation process used in the paper?"
query7 = "How many pdf data were collected from the USA ?"
query8 = "What is the DocLLM architecture ?"

In [44]:
queries = [
    query1,
    query2,
    query3,
    query4,
    query5,
    query7,
    query8,
]

for i, query in enumerate(queries):
    print(f"Example {i+1}: Query->", query)
    print(
        ".." * 50,
    )
    print("Retrieved document:")

    retrieved_documents = retriever.get_relevant_documents(query)
    reranked_documents = rerank_docs(reranker_model, query, retrieved_documents)

    print("--" * 50)
    print(reranked_documents[0][0].page_content)
    print("--" * 50)
    print("metadata:", reranked_documents[0][0].metadata)
    print("==" * 50, "\n\n")

NameError: name 'q' is not defined

In [9]:
results = db.similarity_search_with_relevance_scores(query1, k=1)
print(query1)
pretty_print_docs(results)

Where was the agriculture dataset collected for the USA?

0.6560989752855699

Document(
    page_content='3 Dataset Overview\n\nThis study evaluates fine-tuned and RAG-enhanced language models using 
context-related questions and answers originated datasets from three major crop producer countries: USA, Brazil and
India. In our case, we are using agriculture as the industrial setting as an example. Available data varied 
considerably in format and content, ranging from regulatory documents, scientific reports, agronomic exams, to 
knowledge databases. In this section, we present each dataset in more detail.\n\n3.1 USA',
    metadata={'source': '/teamspace/studios/this_studio/example_data/2401.08406.pdf'}
)

---------

In [10]:
from sentence_transformers import CrossEncoder

reranker_model = CrossEncoder(model_name="BAAI/bge-reranker-large", max_length=512)


def rerank_docs(query, retrieved_docs):
    query_and_docs = [(query, r.page_content) for r in retrieved_docs]
    scores = reranker_model.predict(query_and_docs)
    return sorted(list(zip(retrieved_docs, scores)), key=lambda x: x[1], reverse=True)

In [33]:
retriever = db.as_retriever(search_kwargs={"k": 10})

In [40]:
query = query8
print(query)

What is the DocLLM architecture ?

In [41]:
results = db.similarity_search_with_score(query, k=1)
for r in results:
    print(r[0])

Document(
    page_content='4.2 Model Setup and Training Details\n\nTable 4 provides key settings and hyperparameters for two
variants of DocLLM: DocLLM-1B, which is based on the Falcon-1B architecture [5], and DocLLM-7B, which is based on 
the Llama2-7B architecture [4]3. DocLLM-1B is composed of 24 layers, each with 16 attention heads and a hidden size
of 1,536. DocLLM-7B comprises 36 layers, 32 heads, and a hidden size of 4,096. Using pre-trained weights as the 
backbone for the text modality, we extend the Falcon-1B and Llama2-7B models by adding the disentangled attention 
and block infilling objective as described in Section 3. For DocLLM-1B, we use a pre-training learning rate of 2 × 
10−4 with 1,000 warmup steps, employing a cosine scheduler, and Adam optimizer [58] with β1 = 0.9, β2 = 0.96 and a 
weight decay of 0.1. For instruction tuning we use a learning\n\n3Since Llama2 does not come with pre-trained 
weights at 1B parameters, we use the Falcon-1B architecture for the smaller\n\nversion of DocLLM.\n\n7',
    metadata={'source': '/teamspace/studios/this_studio/example_data/2401.00908.pdf'}
)

In [42]:
retrieved_documents = retriever.get_relevant_documents(query)
reranked_documents = rerank_docs(query, retrieved_documents)
print(reranked_documents[0])

(
    Document(
        page_content='\n\nDocLLM is constructed upon the foundation of an auto-regressive transformer language 
model [4] following a causal decoder structure. It is composed of stacked transformer blocks, where each block 
contains a multi-head self-attention layer and a fully connected feed forward network. Standard language models are
typically unimodal, accepting only a sequence of text tokens as input. In contrast, DocLLM is a multi-modal system 
that integrates lightweight visual information by utilizing the spatial positions and dimensions of text tokens 
obtained using OCR. Simply augmenting the text with bounding box information via additive positional encoding may 
not capture the intricate relationships between text semantics and spatial layout, especially for visually rich 
documents [10]. Consequently, we treat the spatial information about the text tokens as a distinct modality. In 
particular, we use separate vectors to represent these two modalities and extend the self-attention mechanism of 
the transformer architecture to compute their inter-dependencies in a disentangled manner, as explained in the 
following section. Furthermore, instead of the traditional left-to-right next token prediction during 
self-supervised training, we employ a text infilling objective that better leverages contextual information.',
        metadata={'source': '/teamspace/studios/this_studio/example_data/2401.00908.pdf'}
    ),
    0.9895678
)